In [1]:
import pandas as pd
import numpy as np
import time 
import os.path
import datetime
from geopy.distance import geodesic  #计算经纬度点的距离函数


## 一些全局变量

In [2]:
Train_beginDate='20170102'
Train_endDate='20170131'  #临时测试值
#Train_endDate='20170312'  #正式值

car_type_map={'ecar':0, 'rcar':1}  #0-ecar, 1-rcar

## 取日期列表

In [3]:
def dateRange(beginDate, endDate):
    dates = []
    weeks = []
    dt = datetime.datetime.strptime(beginDate, "%Y%m%d")
    date = beginDate[:]
    while date <= endDate:
        dates.append(date)
        weeks.append(dt.weekday())
        dt = dt + datetime.timedelta(1)
        date = dt.strftime("%Y%m%d")
    return dates,weeks



In [4]:
days,weeks=dateRange(Train_beginDate, Train_endDate)
print(len(days))

30


## 读取数据
文件名：BOT_data_ecar_20170102_20170102_part0.csv  
文件名说明：  
1) 每天的数据分3个文件  
2) 文件名格式：前辍_日期_日期_part{n}.csv ,n=0,1,2  


In [5]:
# 导入测试信息数据
grid_msg = pd.read_csv("data\初赛B榜测试信息_加工.csv")
grid_msg.sort(columns=['grid_id'], inplace=True)
grid_msg.head()

E:\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  This is separate from the ipykernel package so we can avoid doing imports until


,grid_id,latitude_begin,latitude_end,longitude_begin,longitude_end
0,51,31.195,31.200,121.320,121.325
1,52,31.180,31.185,121.330,121.335
2,53,31.175,31.180,121.335,121.340
3,54,31.215,31.220,121.415,121.420
4,55,31.185,31.190,121.505,121.510


In [6]:
#由于同一条记录可能在不同grid的边界上，为了在不同grid上都记数，扩展grid_id_n的字段
grid_field_name_list=[]
for i in range(grid_msg.shape[0]):
    field_name="grid_id_{}".format(i+1)
    grid_field_name_list.append(field_name)
    
def get_car_info():
    df_tmp=pd.DataFrame(columns=['grid_id','date','time','hour','car_id','lat','lon','weekday'])
    for day in days:
        for car_type in car_type_map:
            for i in range(3):
                start=time.time()

                filename="data\BOT智能汽车技术挑战赛初赛数据集\{}\BOT_data_{}_{}_{}_part{}.csv".format(\
                                               car_type,car_type,day,day,i)
                if not os.path.exists(filename):
                    print("{} not exist".format(filename))
                    continue
                df_ecar = pd.read_csv(filename, usecols=[0,1,2,3]) #导入ecar数据，只取前4列：car_id,date_time,lat,lon

                # 处理时间字段
                temp = pd.DatetimeIndex(df_ecar['date_time'])
                df_ecar['date'] = temp.strftime("%Y%m%d")
                df_ecar['time'] = temp.time
                df_ecar['hour'] = temp.hour
                df_ecar['weekday']=temp.weekday
                df_ecar['car_type']=car_type_map[car_type]

                #数据清洗，删除非8到23点的数据,前后多取1小时数据
                if day == '20170102':  #1月2号有1月1号的数据
                    data_tmp=df_ecar[(df_ecar['hour']<=23) & (df_ecar['hour']>=8) & (df_ecar['date']==day)]
                else:
                    data_tmp=df_ecar[(df_ecar['hour']<=23) & (df_ecar['hour']>=8)]

                #数据整理，经纬度转成网格ID
                for field_name in grid_field_name_list:
                    data_tmp[field_name]=0

                #grid_id_n置n
                for j in range(grid_msg.shape[0]):
                    row=grid_msg.iloc[j]  
                    grid_id=row['grid_id'].astype(np.int32)
                    field_name=grid_field_name_list[j]

                    data_tmp.loc[(data_tmp.lat >= row['latitude_begin']) & (data_tmp.lat <= row['latitude_end'])\
                        &(data_tmp.lon >= row['longitude_begin']) & (data_tmp.lon <= row['longitude_end']), [field_name]]=grid_id


                #按grid_id生成临时dataFrame,再合并
                for j in range(grid_msg.shape[0]):
                    grid_field_name = grid_field_name_list[j]

                    cloums=[grid_field_name,'date','time','hour','car_id','lat','lon','weekday','car_type']
                    #取grid_id_n大于0的记录，即标记为第n个grid的记录
                    df_by_grid=data_tmp[data_tmp[grid_field_name] > 0][cloums] 
                    
                    #改grid_id_n列名为grid_id
                    df_by_grid.rename(columns={grid_field_name:'grid_id'}, inplace=True) 

                    df_tmp = pd.concat([df_tmp, df_by_grid], axis=0)

                print(filename, " get grid id spended time:", time.time()-start)
                print(df_tmp.shape)
            
            #print(data_tmp.tail())
    return df_tmp
        

In [7]:
start=time.time()
car_data=get_car_info()


E:\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda2\envs\py35\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170102_20170102_part0.csv  get grid id spended time: 17.62700843811035
(3501, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170102_20170102_part1.csv  get grid id spended time: 15.140865802764893
(7001, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170102_20170102_part2.csv  get grid id spended time: 14.329819679260254
(9678, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170102_20170102_part0.csv  get grid id spended time: 67.48285984992981
(67084, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170102_20170102_part1.csv  get grid id spended time: 64.54269170761108
(113034, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170102_20170102_part2.csv  get grid id spended time: 67.24584627151489
(164536, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170103_20170103_part0.csv  get grid id spended time: 25.201441526412964
(170080, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170103_20170103_part1.csv  get grid id spended time: 25.490457773208618

data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170112_20170112_part2.csv  get grid id spended time: 112.41442251205444
(2232539, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170113_20170113_part0.csv  get grid id spended time: 94.57218527793884
(2238555, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170113_20170113_part1.csv  get grid id spended time: 98.0717945098877
(2244618, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170113_20170113_part2.csv  get grid id spended time: 94.71837019920349
(2250889, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170113_20170113_part0.csv  get grid id spended time: 144.01516199111938
(2325264, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170113_20170113_part1.csv  get grid id spended time: 146.4393756389618
(2390945, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170113_20170113_part2.csv  get grid id spended time: 167.5275821685791
(2460166, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170114_20170114_part0.csv  get grid id spended time: 98.925

data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170123_20170123_part0.csv  get grid id spended time: 251.20136785507202
(4403628, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170123_20170123_part1.csv  get grid id spended time: 250.37932085990906
(4453203, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170123_20170123_part2.csv  get grid id spended time: 256.97769832611084
(4502718, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170124_20170124_part0.csv  get grid id spended time: 217.45343780517578
(4509662, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170124_20170124_part1.csv  get grid id spended time: 216.96140956878662
(4518242, 9)
data\BOT智能汽车技术挑战赛初赛数据集\rcar\BOT_data_rcar_20170124_20170124_part2.csv  get grid id spended time: 210.45503735542297
(4525875, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170124_20170124_part0.csv  get grid id spended time: 256.1736524105072
(4578120, 9)
data\BOT智能汽车技术挑战赛初赛数据集\ecar\BOT_data_ecar_20170124_20170124_part1.csv  get grid id spended time: 2

In [8]:
print(" get car_data spended time:", time.time()-start)

 get car_data spended time: 27923.287336826324


In [9]:
car_data.to_csv("B_train_data_all_car_{}_{}.csv".format(Train_beginDate,Train_endDate), index=None)

In [10]:
#通过groupby统计,到car_id一级
group_data=car_data.groupby(['grid_id','date','hour','car_id'],as_index=False).count()
filename="B_train_data_group_by_car_{}_{}.csv".format(Train_beginDate,Train_endDate)
group_data.to_csv(filename, index=None)



In [11]:
#通过groupby统计,到hour一级
df_cnt=group_data.groupby(['grid_id','date','hour'],as_index=False).count().ix[:,0:4]
df_cnt.rename(columns={'car_id':'car_number'}, inplace=True)
filename="B_train_data_group_by_hour_{}_{}.csv".format(Train_beginDate,Train_endDate)
df_cnt.to_csv(filename, index=None)

In [12]:
#通过drop_duplicates统计
'''
drop_data=car_data.drop_duplicates(subset=["grid_id","date", "hour", "car_id"]).ix[:,[0,1,3,4]]
drop_data.head()
drop_cnt=drop_data.groupby(['grid_id','date','hour'],as_index=False).count()
drop_cnt.head()
'''

'\ndrop_data=car_data.drop_duplicates(subset=["grid_id","date", "hour", "car_id"]).ix[:,[0,1,3,4]]\ndrop_data.head()\ndrop_cnt=drop_data.groupby([\'grid_id\',\'date\',\'hour\'],as_index=False).count()\ndrop_cnt.head()\n'

In [14]:
#通过groupby统计,到car_type一级
group_data=car_data.groupby(['grid_id','date','hour','car_type','car_id'],as_index=False).count()
filename="B_train_data_group_by_car_type_{}_{}.csv".format(Train_beginDate,Train_endDate)
group_data.to_csv(filename, index=None)


In [ ]:
#通过groupby统计,到hour一级
df_cnt=group_data.groupby(['grid_id','date','hour','car_type'],as_index=False).count().ix[:,0:4]
df_cnt.rename(columns={'car_id':'car_number'}, inplace=True)
filename="B_train_data_group_by_hour_car_type_{}_{}.csv".format(Train_beginDate,Train_endDate)
df_cnt.to_csv(filename, index=None)